statistica: numero di pixel nell'originale, numero di stati nel modello
minimizzato, tempi di model-checking con voxlogica sull'originale vs.
con graphlogica sul LTS minimizzato (anche se non possiamo visualizzare
il risultato, ma sara' future work).


In [19]:
import subprocess 
import time
import glob
import os
from pathlib import Path
import shutil
import pandas as pd
import tempfile

def run(obj):
    start = time.perf_counter()
    result = subprocess.run(obj["args"],capture_output=True,text=True)
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        result.check_returncode()
    return { "delta": time.perf_counter() - start, "label": obj["label"],"output": result.stdout,"error": result.stderr, "return_code": result.returncode }

converter_exe_rel="../../../releases/GraphLogicA_0.4.1_linux-x64/GraphLogicA"
converter_exe = Path(converter_exe_rel).absolute().as_posix()
minimizer_exe = shutil.which("ltsconvert")
voxlogica_exe = "/home/VoxLogicA/binaries/VoxLogicA_1.0-experimental_linux-x64/VoxLogicA"
output="output"
shutil.rmtree(output,ignore_errors=True)
os.makedirs(output,exist_ok=True)
images = glob.glob("test-images/*.png")

def mk_df(results,delta_label):
    return pd.DataFrame(results).set_index("label").rename(columns={"delta": delta_label}).drop(columns=["output","error","return_code"])

In [20]:
def converter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix()], "label": label }
    #return (run(label,args))

converter_df = mk_df([ run(converter(image)) for image in images ],"conversion")


In [21]:
def minimizer(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    return { "args": [minimizer_exe,"-ebranching-bisim",d_path.as_posix(),m_path.as_posix()], "label": label }

minimizer_df = mk_df([ run(minimizer(image)) for image in images ],"minimization")


CalledProcessError: Command '['/bin/ltsconvert', '-ebranching-bisim', 'output/maze-128.aut', 'output/maze-128_min.aut']' returned non-zero exit status 1.

In [22]:
def fakeConverter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix(),"--fakeconversion"], "label": label }
    #return (run(label,args))

fakeConverter_df = mk_df([ run(fakeConverter(image)) for image in images ],"fakeConversion")


In [23]:
def mazeSpecification(path):
    return f'''
    load img = "{path}"
    let colour(img,r,g,b) = (red(img) =. r) & (green(img) =. g) & (blue(img) =. b)
    let w = colour(img,255,255,255)
    let b = colour(img,0,0,255)
    let g = colour(img,0,255,0)
    let form = (N g) & through(through(w,N b)
    print "vol" volume(form)
    '''

def modelChecker(image):
        path = Path(image)
        spec = mazeSpecification(image)
        fname = tempfile.NamedTemporaryFile().name
        f = open(fname, "w")
        f.write(spec)
        f.close()
        return { "args": [voxlogica_exe,fname], "label": path.name }

modelChecker_df = mk_df([ run(modelChecker(image)) for image in images ],"modelCheckingFull")


CalledProcessError: Command '['/home/VoxLogicA/binaries/VoxLogicA_1.0-experimental_linux-x64/VoxLogicA', '/tmp/tmppgn6ucq2']' returned non-zero exit status 1.

In [18]:
[ run(modelChecker(image)) for image in images[0:1] ]

[{'delta': 0.3676561531610787,
  'label': 'maze-128.png',
  'output': "[        74ms] [info] Parsing input...\n[       122ms] [info] Parse error: Error in /tmp/tmp0y9giqyj: Ln: 7 Col: 24\n                          let form = (N g) & through(through(w,N b)\n                                             ^\n                      Expecting: expression, operator arguments list or '//'\n                      \n[       122ms] [fail] exiting.\n",
  'error': '',
  'return_code': 1}]

In [15]:
df = converter_df.join(fakeConverter_df).join(minimizer_df).join(modelChecker_df)
df

,conversion,fakeConversion,minimization,modelCheckingFull
label,,,,
maze-128.png,0.369485,0.341041,0.051617,0.339960
maze-256.png,0.518756,0.325627,0.113428,0.340637
maze-512.png,0.705163,0.337827,0.431031,0.317685
maze-1024.png,1.577798,0.390455,1.702916,0.322243
maze-2048.png,5.257559,0.558897,6.886114,0.370989
maze-4096.png,20.364001,1.185022,28.500917,0.341592
